In [1]:
import sqlite3
#connect to database
conn = sqlite3.connect('libray.db')
cursor = conn.cursor()

In [2]:
#create book table
cursor.execute('''CREATE TABLE IF NOT EXISTS Books (
                   BookID TEXT PRIMARY KEY,
                   Title TEXT,
                   Author TEXT,
                   ISBN TEXT,
                   Status TEXT
                )''')

In [3]:
#create user table
cursor.execute('''CREATE TABLE IF NOT EXISTS Users (
                    UserID TEXT PRIMARY KEY,
                    name TEXT,
                    email TEXT)''') 

In [4]:
#create reservation table
cursor.execute('''CREATE TABLE IF NOT EXISTS Reservations (
                    ReservationID TEXT PRIMARY KEY,
                    BookID TEXT,
                    UserID TEXT,
                    ReservationDate TEXT)''')

In [5]:
def add_book():
    book_id = input("Please enter book id: ")
    title = input("Please enter book title: ")
    author = input("Please enter book author: ")
    isbn = input("Please enter book isbn: ")
    status = input("Please enter book status: ")
    cursor.execute('''INSERT INTO Books (BookID, Title, Author, ISBN, Status) VALUES (?, ?, ?, ?, ?)''', (book_id, title, author, isbn, status))
    conn.commit()
    print("Book added successfully")

In [16]:
def find_book_details():
    book_id = input("Please enter book id: ")
    cursor.execute('''SELECT Books.BookID, Books.Title, Books.Author, Books.ISBN, Books.Status,
                      Users.UserID, Users.Name, Users.Email, Reservations.ReservationID, Reservations.ReservationDate
                      FROM Books
                      LEFT JOIN Reservations ON Books.BookID = Reservations.BookID
                      LEFT JOIN Users ON Reservations.UserID = Users.UserID
                      WHERE Books.BookID = ?''', (book_id,))
    result = cursor.fetchone()
    if result:
        print("Book Details:")
        print("Book ID:", result[0])
        print("Title:", result[1])
        print("Author:", result[2])
        print("ISBN:", result[3])
        print("Status:", result[4])
        
        if result[4] == 'reserved':
            print("Reserved By:")
            print("User ID:", result[5])
            print("Name:", result[6])
            print("Email:", result[7])
            print("Reservation ID:", result[8])
            print("Reservation Date:", result[9])

    else:
        print("Book not found.")

In [7]:
# Find a book’s reservation status based on the BookID, Title, UserID, and ReservationID.
def find_book_reservation_status():
    reservation_input = input("Please enter book id (starts with LB), title, user id(starts with LU) or reservation id (starts with LR): ")
    if reservation_input.startswith("LB"):#BOOK ID SEARCH
        cursor.execute("SELECT Status FROM Books WHERE BookID = ?", (reservation_input,))
        result = cursor.fetchone()
        if result:
            print("Reservation Status of book: ",reservation_input)
            print("Book Status: ",result[0])
        else:
            print("Book not found.")
    elif reservation_input.startswith("LU"):#USER ID SEARCH
        cursor.execute("SELECT Books.BookID, Books.Status FROM Books LEFT JOIN Reservations ON Books.BookID = Reservations.BookID WHERE Reservations.UserID = ?", (reservation_input,))
        result = cursor.fetchall()
        if result:
            print("Reservation Status of book: ",reservation_input)
            print("Book Status: ",result[1])
        else:
            print("Book not found.")
    elif reservation_input.startswith("LR"):#RESERVATION ID SEARCH
        cursor.execute("SELECT Books.BookID, Books.Status FROM Books LEFT JOIN Reservations ON Books.BookID = Reservations.BookID WHERE Reservations.ReservationID = ?", (reservation_input,))
        result = cursor.fetchall()
        if result:
            print("Reservation Status of book: ",reservation_input)
            print("Book Status: ",result[1])
        else:
            print("Book not found.")
    else:#TITLE SEARCH
        cursor.execute("SELECT Books.BookID, Books.Status FROM Books WHERE Books.Title = ?", (reservation_input,))
        result = cursor.fetchall()
        if result:
            print("Reservation Status of book: ",reservation_input)
            print("Book Status: ",result[1])


In [8]:
#find all the book in the database
def find_all_books():
    cursor.execute('''SELECT Books.BookID, Books.Title, Books.Author, Books.ISBN, Books.Status,
                    Users.UserID, Users.Name, Users.Email, Reservations.ReservationID, Reservations.ReservationDate
                    FROM Books
                    LEFT JOIN Reservations ON Books.BookID = Reservations.BookID
                    LEFT JOIN Users ON Reservations.UserID = Users.UserID''')
    results = cursor.fetchall()
    if results:
        print("All Books:")
        for result in results:
            print("Book ID: ", result[0])
            print("Title: ", result[1])
            print("Author: ", result[2])
            print("ISBN: ", result[3])
            print("Status: ", result[4])
            if result[5]:
                print("Reserved By:")
                print("User ID: ", result[5])
                print("Name: ", result[6])
                print("Email: ", result[7])
                print("Reservation ID: ", result[8])
                print("Reservation Date: ", result[9])
        print("----------------------------------------")
    else:
        print("No books found")


In [37]:
# Modify / update book details based on its BookID
def update_book_details():
    book_id = input("Please enter book id: ")
    cursor.execute('''SELECT Books.BookID, Books.Title, Books.Author, Books.ISBN, Books.Status,
                    Users.UserID, Users.Name, Users.Email, Reservations.ReservationID, Reservations.ReservationDate
                    FROM Books
                    LEFT JOIN Reservations ON Books.BookID = Reservations.BookID
                    LEFT JOIN Users ON Reservations.UserID = Users.UserID
                    WHERE Books.BookID = ?''', (book_id,))
    results = cursor.fetchall()
    result = results[0]
    if result:
        print("Current Book Details:")
        print("Title:", result[1])
        print("Author:", result[2])
        print("ISBN:", result[3])
        print("Status:", result[4])
        print("Reserved By:", result[6])
        print("User ID:", result[5])
        print("Email:", result[7])
        print("Reservation ID:", result[8])
        print("Reservation Date:", result[9])
        while True:
            print("Which part do you want to modify: ")
            print("1.Title")
            print("2.Author")
            print("3.ISBN")
            print("4.Status")
            print("5.User Name")
            print("6.Email")
            print("7.Reservation ID")
            print("8.Reservation Date")
            print("9.Exit")
            choice = input("Please enter your choice: ")
            if choice == "1":
                Title = input("Please enter book title: ")
                cursor.execute('''UPDATE Books SET Title = ?, Author = ?, ISBN = ?, Status = ? WHERE BookID = ?''', ( Title, result[2], result[3], result[4], book_id))
            elif choice == "2":
                Author = input("Please enter book author: ")
                cursor.execute('''UPDATE Books SET Title = ?, Author = ?, ISBN = ?, Status = ? WHERE BookID = ?''', ( result[1], Author, result[3], result[4], book_id))
            elif choice == "3":
                ISBN = input("Please enter book isbn: ")
                cursor.execute('''UPDATE Books SET Title = ?, Author = ?, ISBN = ?, Status = ? WHERE BookID = ?''', ( result[1], result[2], ISBN, result[4], book_id))
            elif choice == "4":
                status = input("Please enter book status: ")
                cursor.execute('''UPDATE Books SET Title = ?, Author = ?, ISBN = ?, Status = ? WHERE BookID = ?''', ( result[1], result[2], result[3], status, book_id))
            elif choice == "5":
                user_name = input("Please enter user name: ")
                cursor.execute('''UPDATE Users SET Name = ?, Email = ? WHERE UserID = ?''', (user_name, result[7], result[5]))
            elif choice == "6":
                email = input("Please enter user email: ")
                cursor.execute('''UPDATE Users SET Name = ?, Email = ? WHERE UserID = ?''', (result[6], email, result[5]))
            elif choice == "7":
                reservation_id = input("Please enter reservation id: ")
                cursor.execute('''UPDATE Reservations SET ReservationID = ?, ReservationDate = ? WHERE BookID = ?''', (reservation_id, result[9], book_id))
            elif choice == "8":
                reservation_date = input("Please enter reservation date: ")
                cursor.execute('''UPDATE Reservations SET ReservationID = ?, ReservationDate = ? WHERE BookID = ?''', (result[8], reservation_date, book_id))
            elif choice == "9":
                break
            # cursor.execute('''UPDATE Books SET Title = ?, Author = ?, ISBN = ?, Status = ? WHERE BookID = ?''', ( Title, Author, ISBN, status, book_id))
            # cursor.execute('''UPDATE Users SET Name = ?, Email = ? WHERE UserID = ?''', (user_name, email, result[5]))
            # cursor.execute('''UPDATE Reservations SET ReservationID = ?, ReservationDate = ? WHERE BookID = ?''', (reservation_id, reservation_date, book_id))
            conn.commit()
            print("Book updated successfully")
        else:
            print("No modification made")
    else:
        print("Book not found.")
        

In [10]:
#Delete a book based on its BookID
def delete_book():
    book_id = input("Please enter book id: ")
    cursor.execute('''SELECT * FROM Books WHERE BookID = ?''', (book_id,))
    result = cursor.fetchone()
    if result:
        print("Current Book Details:")
        print("Title:", result[1])
        print("Author:", result[2])
        print("ISBN:", result[3])
        print("Status:", result[4])
        choice = input("Do you want to delete the book? (y/n): ")
        if choice == "y":
            cursor.execute('''DELETE FROM Books WHERE BookID = ?''', (book_id,))
            conn.commit()
            print("Book deleted successfully")
        else:
            print("No modification made")
    else:
        print("Book not found.")

In [11]:
#main function
def main():
    while True:
        print("Welcome to the Library Management System")
        print("1. Add a book")
        print("2. Find a book's detail based on BookID")
        print("3. Find a book’s reservation status based on the BookID, Title, UserID, and ReservationID")
        print("4. Find all the books in the database")
        print("5. Modify / update book details based on its BookID")
        print("6. Delete a book based on its BookID")
        print("7. Exit")
        choice = input("Please enter your choice: ")
        if choice == "1":
            add_book()
        elif choice == "2":
            find_book_details()
        elif choice == "3":
            find_book_reservation_status()
        elif choice == "4":
            find_all_books()
        elif choice == "5":
            update_book_details()
        elif choice == "6":
            delete_book()
        elif choice == "7":
            break
        else:
            print("Invalid choice")
            
        

In [39]:
if __name__ == "__main__":
    main()

Welcome to the Library Management System
1. Add a book
2. Find a book's detail based on BookID
3. Find a book’s reservation status based on the BookID, Title, UserID, and ReservationID
4. Find all the books in the database
5. Modify / update book details based on its BookID
6. Delete a book based on its BookID
7. Exit
Current Book Details:
Title: I love you
Author: pavolos
ISBN: 999
Status: reserved
Reserved By: Yuepeng
User ID: 202201223
Email: li789@gmail.com
Reservation ID: 003
Reservation Date: 2023-09-21
Which part do you want to modify: 
1.Title
2.Author
3.ISBN
4.Status
5.User Name
6.Email
7.Reservation ID
8.Reservation Date
9.Exit
Book updated successfully
Which part do you want to modify: 
1.Title
2.Author
3.ISBN
4.Status
5.User Name
6.Email
7.Reservation ID
8.Reservation Date
9.Exit
Welcome to the Library Management System
1. Add a book
2. Find a book's detail based on BookID
3. Find a book’s reservation status based on the BookID, Title, UserID, and ReservationID
4. Find all 

### Bonus

In [20]:
import nltk
from nltk.corpus import gutenberg
from gensim import corpora, models
import pyLDAvis.gensim

In [21]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\admib'\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [22]:
# Load the text
raw_text = gutenberg.raw('carroll-alice.txt')

# Tokenize the text into sentences
sentences = nltk.sent_tokenize(raw_text)

# Tokenize each sentence into words
words = [nltk.word_tokenize(sentence) for sentence in sentences]

# Remove stopwords and punctuation, and convert to lowercase
stopwords = nltk.corpus.stopwords.words('english')
words = [[word.lower() for word in sentence if word.isalpha() and word.lower() not in stopwords] for sentence in words]

In [23]:
# Create a dictionary
dictionary = corpora.Dictionary(words)

# Create a corpus
corpus = [dictionary.doc2bow(sentence) for sentence in words]

In [24]:
# Train the LDA model
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

In [26]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)